# openEO Deployment Validation

## Setup

In [ ]:
import openeo
import xarray
import matplotlib.pyplot as pyplot

## Connect to the openEO backend

In [ ]:
connection = openeo.connect(
    url="openeo.test.eoepca.org",
)
connection.authenticate_oidc();

In [ ]:
print("Visit this backend through Web Editor:\n" + connection.web_editor(anonymous=True))

## Exploration

Available collections:

In [ ]:
connection.list_collection_ids()[:10]

Available processes:

In [ ]:
[p["id"] for p in connection.list_processes()[:10]]

## Simple processing

In [ ]:
process_graph = {
    "add": {
        "process_id": "add",
        "arguments": {"x": 3, "y": 5},
        "result": True,
    },
}
connection.execute(process_graph)

## Raster processing

Raster processing with built-in dummy collection (which does not require direct access to actual EO data).

In [ ]:
cube = connection.load_collection(
    "TestCollection-LonLat16x16",
    temporal_extent="2024-09",
    spatial_extent={"west": 3, "south": 51, "east": 5, "north": 53},
    bands=["Longitude", "Latitude", "Day"],
)

In [ ]:
%%time
cube.download("openeo-lonlat1616.nc")

Inspect downloaded results:

In [ ]:
ds = xarray.load_dataset("openeo-lonlat1616.nc")
ds;

In [ ]:
fig, axes = pyplot.subplots(ncols=2, figsize=(7, 3), dpi=75, sharey=True)

ds["Longitude"].isel(t=0).plot.imshow(ax=axes[0])
ds["Latitude"].isel(t=0).plot.imshow(ax=axes[1]);

## Real raster processing with `load_stac`

`load_stac` is an standard openEO process that allows to load EO data that is hosted externally through the STAC standard.

In [ ]:
stac_url = "https://raw.githubusercontent.com/Open-EO/openeo-geopyspark-driver/refs/heads/master/docker/local_batch_job/example_stac_catalog/collection.json"

cube = connection.load_stac(
    url=stac_url,
    temporal_extent=["2023-06-01", "2023-06-09"],
    spatial_extent={"west": 5.07, "south": 51.215, "east": 5.08, "north": 51.22},
    bands=["B04", "B03", "B02"],
)

Simple "band math" based processing, to calculate some NDVI-like index:

In [ ]:
b04 = cube.band("B04")
b02 = cube.band("B02")
n42 = (b04 - b02) / (b04 + b02)

Trigger processing and download result synchronously:

In [ ]:
%%time
n42.download("openeo-load_stac-n42-sync.nc")

Inspect result:

In [ ]:
ds = xarray.load_dataset("openeo-load_stac-n42-sync.nc")
ds;

In [ ]:
fig, ax = pyplot.subplots(dpi=75, figsize=(5, 5))
ds["var"].isel(t=0).plot.imshow(ax=ax, cmap="viridis");